# Importer les bibliothèques

In [74]:
import pandas as pd
import numpy as np
import re

# Ouvrir fichier csv : bien identifier le separateur

In [75]:
file = 'openaq.csv'
df = pd.read_csv(file, sep = ';')

# Apercu du fichier

In [76]:
df

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
0,DE,Manfred Lotz,"Cottbus, W.-Külz-Str.","51.75413,14.325549",PM2.5,EEA Germany,µg/m³,21.7,2016-12-21T12:00:00+01:00,Germany
1,ES,Castellón/Castelló,ES1969A,"39.8236111095,-0.24444444",NO2,EEA Spain,µg/m³,1.0,2021-07-13T08:00:00+02:00,Spain
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.0,2021-08-06T11:00:00+02:00,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.0,2021-08-06T11:00:00+02:00,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.0,2021-08-06T11:00:00+02:00,Spain
...,...,...,...,...,...,...,...,...,...,...
40446,US,BERNALILLO,WESTSIDE - 9 Mile,"35.064098,-106.761292",PM10,AirNow,µg/m³,10.0,2020-07-17T14:00:00+02:00,United States
40447,US,Asheville,Skyland DRR,"35.481861,-82.509861",SO2,AirNow,ppm,0.0,2020-07-02T12:00:00+02:00,United States
40448,US,San Antonio,San Antonio IH35 C10,"29.5294,-98.39139",PM2.5,AirNow,µg/m³,6.0,2020-06-26T01:00:00+02:00,United States
40449,US,Cleveland-Elyria-Mentor,Cleveland Near Road,"41.4409,-81.4949",PM2.5,AirNow,µg/m³,15.3,2020-05-26T16:00:00+02:00,United States


In [77]:
df.dtypes

Country Code      object
City              object
Location          object
Coordinates       object
Pollutant         object
Source Name       object
Unit              object
Value            float64
Last Updated      object
Country Label     object
dtype: object

## Supprimer les données qui ne concernent pas la journée du 06-08-2021.

### On divise la colonne last updated pour ne conserver que la date (pas nécessaire apparemment, contains suffit)

In [78]:
df[['Last Updated','Date']] = df['Last Updated'].str.split('T',expand=True)

### On supprime la colonne obtenue par le split

In [79]:
df = df.drop(columns=['Date'])

### On recherche les lignes qui contiennent la date précise :

In [80]:
df = df[df['Last Updated'].str.contains("2021-08-06")]

In [81]:
df

,Country Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
2,ES,Valencia/València,ES1885A,"39.4802777695,-0.33638888",SO2,EEA Spain,µg/m³,5.000,2021-08-06,Spain
3,ES,Alicante/Alacant,ES2008A,"37.9911111095,-0.69",PM2.5,EEA Spain,µg/m³,9.000,2021-08-06,Spain
4,ES,Alicante/Alacant,ES1968A,"38.3511111095,-0.51388888",PM10,EEA Spain,µg/m³,1.000,2021-08-06,Spain
5,ES,Valencia/València,ES1911A,"39.7052777695,-0.33666666",CO,EEA Spain,µg/m³,100.000,2021-08-06,Spain
6,ES,Valencia/València,ES1185A,"39.6672222195,-0.23472222",PM10,EEA Spain,µg/m³,13.000,2021-08-06,Spain
...,...,...,...,...,...,...,...,...,...,...
40408,TH,Bangkok,"Din Dang, Bangkok","13.7619223,100.558606",PM10,Air4Thai,µg/m³,42.000,2021-08-06,Thailand
40409,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",SO2,Air4Thai,ppm,0.000,2021-08-06,Thailand
40410,TH,Samut Prakan,"City Hall, Samut Prakan","13.599149,100.597345",NO2,Air4Thai,ppm,0.006,2021-08-06,Thailand
40411,TH,Chachoengsao,Municipality Office Tungsadao,"13.588554,101.286359",PM10,Air4Thai,µg/m³,37.000,2021-08-06,Thailand


### Comment être certain que toutes les dates sont au bon format ?

## Supprimer les données qui sont illisibles et pour lesquelles on ne comprend pas la ville concernée.

In [63]:
# source : https://stackoverflow.com/questions/36340627/remove-non-ascii-characters-from-pandas-column

In [82]:
df = df.rename(columns={'Country Code': 'Code'})

In [84]:
df.loc[df.Code=='CN', :]

,Code,City,Location,Coordinates,Pollutant,Source Name,Unit,Value,Last Updated,Country Label
496,CN,郑州市,上街区政府,"34.803229,113.30952",CO,ChinaAQIData,µg/m³,800.0,2021-08-06,China
497,CN,伊春市,中心小学,"47.4346,128.4741",PM10,ChinaAQIData,µg/m³,14.0,2021-08-06,China
498,CN,滨州市,中海酒店,"37.3956,117.9378",O3,ChinaAQIData,µg/m³,189.0,2021-08-06,China
499,CN,北京市,丰台小屯,"39.8768,116.2545",CO,ChinaAQIData,µg/m³,700.0,2021-08-06,China
500,CN,聊城市,九州社区,"36.4361,116.0445",CO,ChinaAQIData,µg/m³,900.0,2021-08-06,China
...,...,...,...,...,...,...,...,...,...,...
40012,CN,哈尔滨市,呼兰师专,"45.9855,126.595",O3,ChinaAQIData,µg/m³,107.0,2021-08-06,China
40013,CN,嘉兴市,嘉兴学院,"30.7478,120.726",SO2,ChinaAQIData,µg/m³,3.0,2021-08-06,China
40014,CN,长春市,园林处,"43.8805,125.3297",SO2,ChinaAQIData,µg/m³,3.0,2021-08-06,China
40015,CN,江门市,圭峰西,"22.5311,113.0219",CO,ChinaAQIData,µg/m³,700.0,2021-08-06,China


## Pour chaque polluant, vérifier qu'on n'a qu'une seule mesure utilisée. Autrement, supprimer les données qui correspondent à la mesure minoritaire ou moins cohérente.


## Regarder s'il y a des valeurs aberrantes ou des problèmes par rapport à notre objectif final (pouvoir comparer des mesures sur un maximum de pays) et noter ces aspects dans un fichier texte.